# Neuro DS Midterm
This midterm will cover the material that we discussed in the first 5 weeks of class. It will take the methods that we've covered thus far into a new dataset, in order to see if there is something that we can learn from the reults.

# Remember
* Make sure you use `%matplotlib inline` before calling any plots that aren't interactive. When you upload a PDF of the midterm, we *need* to see the plots. **We will deduct points for missing plots.**
* When you're writing code, don't make the code extend beyond the visible code block. **This will push your content outside of the margins of the PDF and we will not be able to see your full answer**. If you need to write more, hit enter and continue on the next line. If you're writing a comment (e.g. answering a conceptual question), then just start a new line.
* You only need to write code in the blocks that contain the text `### STUDENT ANSWER`. There are some blocks with code that we've already written for you to make things easier.
* to submit this homework, **submit both a PDF and the raw jupyter notebook file** to the assignment page on bCourses
* Please review the PDF before submitting it to make sure it looks right. 

# The Dataset
The dataset that we'll use comes from a (different) electrocorticography patient. The subject had intractible epilepsy, which required surgery in order to address. A common problem with neurosurgery is that we don't know *exactly* which regions of the brain are crucial for performing some function. For example, we may know the general region that processes to sounds, but in each individual the location will be a little different.

To deal with this, surgeons often use *localizers* to determine where a particular region is in one person. This surgeon in particular was interested in a region of the brain called the *Fusiform Face Area* (FFA), a well-studied part of the visual system that seems to be particularly responsive to faces. The surgeon would like to make sure not to harm this important region of the brain when doing surgery.

So, the surgeon presented a number of visual stimuli for the patient to look at. They placed a strip of electrodes over the general area where we'd expect the patient's FFA to be (along with a larger grid over the right hemisphere). The subject viewed images of faces as well as objects.

Here are examples of two potential visual stimuli:

Face:
<img src="http://pngimg.com/upload/face_PNG5660.png" alt="face" style="width: 100px;"/>

Object:
<img src="http://i74.photobucket.com/albums/i241/cmucam/Black_Stapler_zpsc3024407.jpg" alt="object" style="width: 100px;"/>


Our task will be to analyze the data collected from these two types of visual stimuli, in order to determine if we can identify the electrode that corresponds to the Fusiform Face Area.

To do this, here is one function that will be useful later on. Make sure to run the cell below so that the function exists, and when you're asked to use it, refer to this function's docstring to understand what parameters to give it.

In [ ]:
import mne
import neurods as nds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datascience as ds
%matplotlib notebook

# Here's a predefined function to make the plot
def plot_four_channels(evoked_dictionary, indices):
    """Plot the evoked activity of four channels.
    
    Parameters
    ----------
    evoked_dictionary : dictionary
        A dictionary mapping an event type string onto the `Evoked` object for that
        event type.
    indices : list of integers, length 4
        A list of integers corresponding to the electrodes you wish to plot.
    """
    # Calculate the amplitude of all included channels
    evoked_list = list(evoked_dictionary.values())
    evoked_data = np.hstack([ii.data[indices] for ii in evoked_list])
    amps = np.abs(evoked_data).max(axis=1)
    
    # Sort the channels so we plot the most active first
    indices = np.array(indices)
    ixs_sorted = np.argsort(amps)[::-1]
    ixs_sorted = indices[ixs_sorted]
    
    # Now do the plotting
    fig, axs = plt.subplots(2, 2, figsize=(10, 10), sharey=True, sharex=True)
    for ix, ax in zip(ixs_sorted, axs.ravel()):
        ix = int(ix)
        fig = mne.viz.plot_compare_evokeds(evoked_dictionary,
                                           picks=[ix], axes=ax, show=False)
    fig.show()

Now on to the data analysis. Let's take a look at our ECoG grid to see where these electrodes are located...

We'll load the raw data that we'll use today...

In [ ]:
# --- Set path ---
path_data = '/home/shared/cogneuro-connector/data/midterm/proc/'
# --- Raw data ---
raw = mne.io.Raw(path_data + 'ecog-raw.fif', preload=True)

# --- Event time information ---
# Or as a datascience table (use whichever you like)
mtime = ds.Table.read_table(path_data + 'meta_time.csv', index_col=0)

# If you want to use a pandas dataframe instead of a datascience `Table`, then
# uncomment the code below and use this:
# mtime = pd.read_csv(path_data + 'meta_time.csv', index_col=0)

# --- Image and layout of the electrode grid ---
im = plt.imread(path_data + 'brain.png')
melec = pd.read_csv(path_data + 'meta_elec.csv', index_col=0)
layout = mne.channels.read_layout('channel_layout.lout', path_data,
                                  scale=False)

And then make a quick plot of all the electrodes for this ECoG patient.

In [ ]:
%matplotlib inline
def plot_ecog_layout(layout, im, textcolor='k'):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(im)
    for _, (ch, _, x, y) in  melec.iterrows():
        ax.scatter(x, y, c='r', s=60)
        ax.annotate(ch, (x, y), (x-5, y-5),
                    rotation=0, fontsize=14, color=textcolor)
    ax.set_axis_off()
    return fig

fig = plot_ecog_layout(layout, im)

In [ ]:
# Don't forget you can switch on interactive mode with:
# %matplotlib notebook

# And inline mode with:
# %matplotlib inline

Most of the grid covers the right hemisphere, however the strip to the bottom left is important. **We think that it might cover the FFA**. We will particularly pay attention to the activity in this strip in order to locate this important region of the brain.

# Loading and plotting raw data
First, we need to get a high-level view of the dataset and drop any channels that look bad.

* Plot the raw data we've loaded above. Scan through the entire session of data.
* There is a noisy channel in there, mark its channel name, and put it in a list.
* Finally, drop the channel from the ecog data.

In [ ]:
%matplotlib notebook

# Interactively browse through the data here
### STUDENT ANSWER


In [ ]:
# Put your noisy channel in a list (and drop it), here:

### STUDENT ANSWER


# Plotting evoked activity for each type of visual stimulus
Now that we've got clean channels, it's time to calculate the evoked activity in response to each type of visual stimulus. The visual stimuli are coded like this:

- Faces : 7
- Objects : 3

We'll use event timing information in order to slice up our data and visualize how the brain responds to the pictures:

* Create an events array that we can use to make an MNE `Epochs` object.  Use the timing information in `mtime`. You can do this by creating the following things:
    * One vector of event onsets (in samples). Note that the event onsets in `mtime` are in seconds, not samples.
    * A vector of 0s that has the same length as the event onsets vector.
    * A vector that is made up of integers, one per event, that represents the integer ID corresponding to the "type" of the event.
* Combine these three things into an array of shape `(n_events, 3)`. (make sure that this array has a dtype of `int`). Remember that you can vertically stack vectors with `np.vstack`, and you can transpose an array with `array.transpose`
* Create a dictionary that maps the event type names (faces and objects) onto unique event type integers (7 and 3)

In [ ]:
### STUDENT ANSWER


# Epochs ERPs
Now that we have the events array, we can create an `Epochs` object and look at the evoked activity.

* Filter the raw data between 1 and 30 Hz
* Using the events array above, turn the `Raw` object into an `Epochs` object.
  * Include times from -.3 to .8 seconds
  * Include a baseline from -.3 to -.05 seconds (use the `baseline` parameter)
  * Make sure to include the event ID dictionary you created above

In [ ]:
### STUDENT ANSWER


In case you weren't able to create the epochs data above, here we are loading the correct epochs data below. You can use this to continue. Below we have named the variable `epochs`, but make sure that you name it whatever works with the code that you've written.

In [ ]:
epochs = mne.read_epochs(path_data + 'epochs_filt-epo.fif')

* Create an MNE `Evoked` object by using the `average` method of the `Epochs` object.
  * Create one `Evoked` object for each event type (for a total of 2 `Evoked` objects)
  * Remember you can take a subset of the epochs belonging to a single event type by passing it as a string to the `epochs` object, e.g. `epochs['my_event_type']`

In [ ]:
### STUDENT ANSWER


Now let's look at the evoked activity:
* Plot a topomap of the evoked activity using `mne.viz.plot_evoked_topo`.
    * Use the layout supplied above as the input to the `layout` parameter,  and the image of the brain given above as the `fig_background` parameter.
    * Remember that you can put multiple `Evoked` objects in a list and pass this as the first argument to the `plot_evoked_topo`. This way it will plot evoked activity from multiple conditions on top of one another.

In [ ]:
%matplotlib inline
### STUDENT ANSWER


Now let's look at the ERPs of some individual channels. To do this, we'll use a function that we created above called `plot_four_channels`, which lets us compare the evoked activity for multiple condition types. We'll plot four channels, making a plot for each one.

* Create a dictionary, where each key is a string corresponding to an event type, and each value is the `Evoked` object for that event type.
* Call it: `evoked_dictionary`

In [ ]:
### STUDENT ANSWER


* Use this dictionary as input to the `plot_four_electrodes` function defined above. It will plot the ERP activity for four pre-selected electrodes. We'll plot the two electrodes over the FFA as well as two nearby electrodes on the temporal lobe.
  * Remember to look at the documentation for `plot_four_electrodes` if you're confused about what inputs it takes.

In [ ]:
%matplotlib inline
ixs_plt = mne.pick_channels(av1.ch_names, ['IO02', 'IO03', 'G08', 'G16'])

### STUDENT ANSWER


* Does it seem like there are differences between these two conditions in any of these electrodes? Why or why not?
* What else could we try in order to determine if an electrode shows a difference in how it responds to visual stimuli?

In [ ]:
### STUDENT ANSWER


# Time-frequency representation
Now we will turn to a more complicated view of the electrode activity: the time-frequency representation.

First, we'll re-load the `Raw` data, because earlier we modified it in-place. By re-loading from disk, we can start fresh:

In [ ]:
raw = mne.io.Raw(path_data + 'ecog-raw.fif', preload=True)

* Use this `Raw` object along with the event timings to create another `Epochs` object from the raw data.

In [ ]:
### STUDENT ANSWER


Again, in case you weren't able to do this correctly, here's the data that you should now have:

In [ ]:
epochs = mne.read_epochs(path_data + 'epochs_tfr-epo.fif')

* Use morlet wavelets to create a time-frequency representation of the `Epochs` data.  **For each event type**:
    * You should use the `tfr_morlet` function
    * Use frequencies from 10 to 150 Hz, with a spacing of 6Hz (use the `np.arange` function.)
    * Use `n_cycles=5`.
    * You should end up with 2 arrays (representing the TFR for each condition). Print the shape of each one.

In [ ]:
### STUDENT ANSWER


* Calculate mean TFR for each condition by averaging across trials
* then use the result to create an `AverageTFR` object for each condition.
  * Remember that `AverageTFR` expects an input of shape `(n_channels, n_frequencies, n_times)`.

In [ ]:
### STUDENT ANSWER


* Now, make a topographic map of the TFR for each condition.
    * Use the `plot_topo` method of the `AverageTFR` object you've created. 
    * Use a baseline of `(None, 0)`
    * set `mode='zscore'`
    * set the vmin/vmax to -3 and 3
    * Use the `picks` for all electrodes (it will be the first parameter to the function)
    * Don't forget to set the image of the brain as the `fig_background`.

In [ ]:
%matplotlib inline
vmin, vmax = -3, 3
mode = 'zscore'
baseline = (None, 0)
picks = range(len(tfr_a_1.ch_names))

### STUDENT ANSWER


* Describe in general what the time-frequency activity looks like in either condition (don't worry about comparing the two yet) across the brain. Which regions of the brain seem to be more active?

In [ ]:
### STUDENT ANSWER


* Now, calculate the difference between the two `AverageTFR` objects (you can directly subtract the two).
* Make a topo plot of this result as well.

In [ ]:
%matplotlib inline
### STUDENT ANSWER


It looks like those electrodes over FFA seem to be doing something. Let's investigate further.

* Make a TFR plot of the channels we plotted earlier (two channels in the strip, and two in the grid) for the difference between the two conditions (you can use the same `ixs_plot` variable that we used above).
  * Use the `plot` method of the `AverageTFR` obects.
  * Set the mode to `'zscore'`, the baseline to `(None, 0)`, and the vmin/vmax to -10 and 10, respectively.
  * Remember the first argument to `plot` is a list of indices, corresponding to which channels you want to plot. You should use the same indices we used earlier to make the ERP comparisons.

In [ ]:
mode = 'zscore'
baseline = (None, 0)
vmin, vmax = -10, 10

### STUDENT ANSWER


* How do the differences between these TFRs compare with the ERPs that we calculated earlier (when we used `plot_compare_evokeds`)?

In [ ]:
### STUDENT ANSWER


To avoid memory getting overloaded, delete the two TFR objects that you created above. These take up too much memory. Below we've shown the code for doing this, but you might need to rename the variables depending on what you called them

In [ ]:
del tfr_face
del tfr_object

# High-frequency amplitude
It looks like the majority of the differences are in the high-frequencies (>40Hz or so). Let's focus on that and look at the effects.

* On the raw data, use the `nds.tfr.extract_amplitude` function to extract linearly-spaced frequencies from 70 to 150 Hz, in steps of 10Hz. (use `np.arange`)
* Then create an `Epochs` object from this high-frequency ampitude using the same event times above.

In [ ]:
### STUDENT ANSWER


In case you were not able to create the epochs and extract high-frequency amplitude above, here is the epochs file that you'll need to finish this section:

In [ ]:
epochs_hfa = mne.read_epochs(path_data + 'epochs_hfa-epo.fif')

* Create an `Evoked` object for each condition type.
* Calculate the difference between the two `Evoked` objects
* Plot this difference as a topo plot on the brain

In [ ]:
%matplotlib inline
### STUDENT ANSWER


It looks like the two electrodes to the bottom left are again showing the only effects. Let's take a look at them.

* Once again, create a dictionary that maps event type strings onto `Evoked` objects.
* Call it `evoked_dictionary_hfa`

In [ ]:
### STUDENT ANSWER


* Use this dictionary as input to the same `plot_four_channels` function we used above. It will plot the evoked activity or high-frequency amplitude.
* You'll also need a list of integers corresponding to the channels we wish to plot, which we'll give below.

In [ ]:
# Here are the electrodes you should plot
ixs_plt = mne.pick_channels(av1.ch_names, ['IO02', 'IO03', 'G08', 'G16'])
%matplotlib inline

### STUDENT ANSWER


* Which electrode(s) are the most responsive to visual stimuli in general?
* Which electrode(s) are the most responsive to faces? Which are most responsive to objects?
* Does this seem like a more or less reliable way to distinguish faces vs. objects, compared with using the raw data (the plot we made earlier in this exam)?

In [ ]:
### STUDENT ANSWER


# Extra credit - Comparing variance
As a bonus, let's compre how noisy the raw ERPs are compared with the evoked high-frequency activity. Below we'll load the `Epochs` objects associated with each one, and take a subset of trials corresponding to faces:

In [ ]:
epochs_hfa = mne.read_epochs(path_data + 'epochs_hfa-epo.fif', preload=True)
epochs_hfa = epochs_hfa['face'].crop(-.2, .5)

epochs_raw = mne.read_epochs(path_data + 'epochs_filt-epo.fif', preload=True)
epochs_raw = epochs_raw['face'].crop(-.2, .5)

* For each `Epochs` object, calculate its mean **and** its standard error across trials. You can use the `nds.stats.standard_error` function.

In [ ]:
### STUDENT ANSWER


* Finally, for the two active electrodes that we plotted above, make a plot showing the mean +/- the standard error for each data type (one for HFA, one for raw data).
  * Plot the raw data with `ax1` and the HFA data with `ax2`.
  * use the `ax.fill_between` function. This will accept a vector of x-values to plot (the `times` in our epochs objects), a vector of lower y-values to plot (the mean - the standard error), and a vector of upper y-values to plot (the mean + the standard error)

In [ ]:
%matplotlib inline
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

### STUDENT ANSWER


* Which type of data seems to have a more clear difference between the two conditions?

In [ ]:
### STUDENT ANSWER
